In [50]:
import torch 
import torch.nn as nn
import numpy as np
from GPT_model import * 
import pickle
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [51]:
market = 'KOSDAQ'
year = '2023'


train_x = np.load(f'./data/{market}/new_normalize_data3/{market}_{year}_train_x_norm_close2close.npy')
train_y = np.load(f'./data/{market}/new_normalize_data3/{market}_{year}_train_y_norm_close2close.npy') 


day_close = []

# day_close_btw = []

ohlc_rate_list = []

last_close_rate  = []


with open(f'./data/{market}/{market}_{year}.pkl', 'rb') as f:
    KOSPI_data = pickle.load(f)
        
    data = KOSPI_data['data']
    valid_ticker = KOSPI_data['valid_ticker']
    dimension = KOSPI_data['dimension']
    features = KOSPI_data['features']
    dates = KOSPI_data['dates']
    
    tickers_num, time_step, features_dim = data.shape
    
    window = 60
    predict_day_after = 20

    for i in tqdm(range(time_step-window),total= time_step-window):
    
        temp = data[:,i:i+window,:].copy()
        
        try : 
            day_close_price = data[:,i+window-1,3]
            predict_day_close = data[:,i+window+predict_day_after,3]
        except : 
            break
        
        day_close.append(day_close_price)
        
        ohlc = data[:,i+window:i+window+predict_day_after,0:4].copy()
        
        ohlc_rate = (ohlc - day_close_price[:, np.newaxis,np.newaxis])/day_close_price[:, np.newaxis,np.newaxis]
        
        ohlc_rate_list.append(ohlc_rate)
        
        
        y = (predict_day_close - day_close_price) / day_close_price
        
        last_close_rate.append(y)

        
        # close_btw = data[:,i+window:i+window+predict_day_after,3]
        # close_rate_btw = (close_btw - day_close_price[:, np.newaxis])/day_close_price[:, np.newaxis]
        # day_close_btw.append(close_rate_btw)

# last_close_rate_np = np.concatenate(last_close_rate, axis=0)

X_train_tensor = torch.tensor(train_x, dtype=torch.float32)
X_train_tensor_reshaped = X_train_tensor.view(len(data),data.shape[1]-window-predict_day_after,train_x.shape[1],features_dim)
X_train_tensor_reshaped_transpose = X_train_tensor_reshaped.permute(1, 0, 2, 3)  

y_train_tensor = torch.tensor(train_y, dtype=torch.float32)
y_train_tensor_reshaped = y_train_tensor.view(len(data),data.shape[1]-window-predict_day_after)
y_train_tensor_reshaped_transpose = y_train_tensor_reshaped.permute(1, 0)  

day_close_tensor = torch.tensor(np.array(day_close), dtype=torch.float32)
ohlc_rate_tensor = torch.tensor(np.array(ohlc_rate_list), dtype=torch.float32)
last_close_rate_tensor = torch.tensor(np.array(last_close_rate), dtype=torch.float32)



print(X_train_tensor_reshaped_transpose.shape)
print(y_train_tensor_reshaped_transpose.shape)
print(day_close_tensor.shape)   
print(ohlc_rate_tensor.shape)
print(last_close_rate_tensor.shape)

dataset = (X_train_tensor_reshaped_transpose,y_train_tensor_reshaped_transpose,day_close_tensor,ohlc_rate_tensor,last_close_rate_tensor)


  0%|          | 0/132 [00:00<?, ?it/s]

torch.Size([112, 1338, 60, 42])
torch.Size([112, 1338])
torch.Size([112, 1338])
torch.Size([112, 1338, 20, 4])
torch.Size([112, 1338])


In [56]:
BALANCE = 100000000

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Define hyperparameters
learning_rate = 1e-4
num_epochs = 10

# Initialize model and optimizer
embed_size = 42  # Number of features in your data
num_layers = 6  # Number of transformer blocks (this is a hyperparameter, you can adjust it)
heads = 6  # Number of attention heads (another hyperparameter)
forward_expansion = 1  # Expansion factor for the feed-forward layer
max_length = 60  # Your sequence length
dropout = 0 # Dropout rate

model = GPT(
    embed_size=embed_size,
    num_layers=num_layers,
    heads=heads,
    forward_expansion=forward_expansion,
    max_length=max_length,
    dropout=dropout
)

model.load_state_dict(torch.load(f'./model/layer6_new/Transformer_layer6_4.pt'))
model.to(device)
model.pe = model.pe.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.BCEWithLogitsLoss() 

In [60]:
BUY_TAX = 0.00015
SELL_TAX = 0.003 + BUY_TAX


ORI_BALANCE = 1000000000
confidence = 0.95

inputs, targets, close_prices,ohlc_rate,last_close_rate = dataset
inputs, targets, close_prices,ohlc_rate,last_close_rate = inputs.to(device), targets.to(device),close_prices.to(device),ohlc_rate.to(device),last_close_rate.to(device)

date = targets.shape[0]
num_ticker = targets.shape[1]
 


hellos = 0 
new_invest = 0 
print(f'--------------------- {market} {year} ---------------------')
model.eval()
with torch.no_grad():
    
    for i in range(date) : 
        correct_predictions = 0 
        if i == 0 : 
            BALANCE = ORI_BALANCE
        else : 
            BALANCE = NEW_BALANCE

        input = inputs[i]
        target = targets[i]
        close_price = close_prices[i]
        ohlc = ohlc_rate[i]
        last_close = last_close_rate[i]


    
        outputs = model(input)
        probabilities = torch.sigmoid(outputs)
        
        real_target = (target > 0.3).float().squeeze()
        predictions = (probabilities > 0.5).float().squeeze()
        
        invest = (probabilities > confidence).view(-1)
        invest_bool = invest.squeeze().bool()

        num_invest = invest.float().sum()
        
        invest_ohlc = ohlc[invest_bool]
        invest_last_close = last_close[invest_bool]
        
        
       
        
        invest_target = target[invest_bool]
        
        
        
        
        # print(invest_ohlc.shape)              # 20일 안의 모든 ohlc rate 
        # print(invest_last_close.shape)        # 20일 뒤의 close rate
        # print(invest_target.shape)            # max값
        # input()
        
        idx_list = []
        

        thres = 0.05

        for day in range(invest_ohlc.shape[1]):
            ohlc_ticker = invest_ohlc[:,day,:]
            max_ohlc_ticker = torch.max(ohlc_ticker,axis=1)
            
            # close_btw = invest_ohlc[:,day,3]
            

            
            thres_idx = torch.nonzero(max_ohlc_ticker.values > thres).squeeze().tolist()

            
            # Convert thres_idx to a list if it's a single integer
            if not isinstance(thres_idx, list):
                thres_idx = [thres_idx]

            # Check and update values
            for idx in thres_idx:
                if idx not in idx_list:
                    invest_last_close[idx] = thres
                    idx_list.append(idx)
                        
            

        new_invest_target_rate = invest_last_close.sum()/num_invest
        
        # print(new_invest_target_rate) 
        # input()
        

        correct_predictions += (invest == real_target).sum().item()
        
        accuracy = correct_predictions/len(real_target) * 100
        
        TP = torch.sum((invest == 1) & (real_target == 1))
        FP = torch.sum((invest == 1) & (real_target == 0))
        
        TP_avg = TP / real_target.sum() * 100
        try : 
            false_positive_indices = torch.nonzero((invest == 1) & (real_target == 0)).squeeze()
            true_positive_indices = torch.nonzero((invest == 1) & (real_target == 1)).squeeze()
            true_positive_rate = target[true_positive_indices].sum()/len(true_positive_indices)
            false_positive_rate = target[false_positive_indices].sum()/len(false_positive_indices)
            
        except : 
            continue
        
        invest = (probabilities > confidence).view(-1)

        num_invest = invest.float().sum()
        hello = last_close[invest.squeeze()].sum()/num_invest
        
        
        buy_BAL = torch.zeros_like(probabilities)
        buy_BAL[invest] = probabilities[invest]
        
        softmax_tensor = F.softmax(buy_BAL, dim=0)
        
        Balance_to_buy_each_ticker = (softmax_tensor * BALANCE).view(-1)
        
        num_stock_can_buy = Balance_to_buy_each_ticker // close_price   # 몫
        remain_bal = (Balance_to_buy_each_ticker % close_price).sum()   # 나머지의 합
        
        buy_tax_bal = (num_stock_can_buy * close_price).sum() * BUY_TAX
        
        NEW_BALANCE = (num_stock_can_buy * close_price * (1+target)).sum()* (1-SELL_TAX) + remain_bal - buy_tax_bal
        profit = (NEW_BALANCE - ORI_BALANCE) / ORI_BALANCE * 100
        hellos += hello
        new_invest += new_invest_target_rate
        total_rate = hellos / (i+1)
        total_new_invest = new_invest / (i+1)
        # print(f'[DAY {dates[i+window]}] signal :{int(num_invest)}/{num_ticker}, profit : {profit:.2f}%, acc : {accuracy:.2f}% TP_acc : {TP_avg:.2f}%, TP_rate : {true_positive_rate*100:.2f}%, FP_rate = {false_positive_rate*100:.2f}%, rate_avg : {hello*100:.2f}%')
        # print(f'[DAY {dates[i+window]}] signal :{int(num_invest)}/{num_ticker}, acc : {accuracy:.2f}%, TP_acc : {TP}/{int(real_target.sum())}, FP_acc : {FP}/{len(real_target) - int(real_target.sum())}, TP_rate : {true_positive_rate*100:.2f}%, FP_rate = {false_positive_rate*100:.2f}%, rate_avg : {hello*100:.2f}%, total_rate : {total_rate*100:.2f}%')
        print(f'[DAY {dates[i+window]}] signal :{int(num_invest)}/{num_ticker}, new_invest_target_rate : {new_invest_target_rate*100:.2f}%, total_new_invest : {total_new_invest*100:.2f}%,  TP_acc : {TP}/{int(real_target.sum())}, FP_acc : {FP}/{len(real_target) - int(real_target.sum())}, TP_rate : {true_positive_rate*100:.2f}%, FP_rate = {false_positive_rate*100:.2f}%, rate_avg : {hello*100:.2f}%, total_rate : {total_rate*100:.2f}%')
    



--------------------- KOSDAQ 2023 ---------------------
[DAY 2023-03-30 00:00:00] signal :285/1338, new_invest_target_rate : 0.65%, total_new_invest : 0.65%,  TP_acc : 42/154, FP_acc : 243/1184, TP_rate : 61.87%, FP_rate = 9.81%, rate_avg : 0.69%, total_rate : 0.69%
[DAY 2023-03-31 00:00:00] signal :295/1338, new_invest_target_rate : -0.27%, total_new_invest : 0.19%,  TP_acc : 53/146, FP_acc : 242/1192, TP_rate : 56.23%, FP_rate = 10.23%, rate_avg : -2.57%, total_rate : -0.94%
[DAY 2023-04-03 00:00:00] signal :289/1338, new_invest_target_rate : 1.31%, total_new_invest : 0.56%,  TP_acc : 42/123, FP_acc : 247/1215, TP_rate : 53.32%, FP_rate = 9.98%, rate_avg : 0.55%, total_rate : -0.45%
[DAY 2023-04-04 00:00:00] signal :287/1338, new_invest_target_rate : 0.39%, total_new_invest : 0.52%,  TP_acc : 48/154, FP_acc : 239/1184, TP_rate : 55.81%, FP_rate = 10.23%, rate_avg : 0.59%, total_rate : -0.19%
[DAY 2023-04-05 00:00:00] signal :295/1338, new_invest_target_rate : 0.77%, total_new_invest 

In [ ]:
avg_rate= total_rate
new_avg_rate = total_new_invest

print(f'annual rate : {((1+avg_rate)**12 - 1) * 100:.2f}%') 
print(f'new annual rate : {((1+new_avg_rate)**12 - 1) * 100:.2f}%') 

annual rate : -22.18%
new annual rate : -11.82%
